# Change the ecoinvent version of the mapping file

In [2]:
import pandas as pd
import bw2data as bd
from mescal import *
import itertools

In [3]:
bd.projects.set_current("ecoinvent3.9")

In [4]:
def load_change_report_annex(v_from, v_to):
    df = pd.read_excel(f"data/Change Report Annex v{v_from} - v{v_to}.xlsx", sheet_name="Qualitative Changes", usecols=[f'Reference Product - {v_from}', f'Activity Name - {v_from}', f'Geography - {v_from}', f'Reference Product - {v_to}', f'Activity Name - {v_to}', f'Geography - {v_to}'])
    df.rename(columns={f'Activity Name - {v_from}': 'Activity Name', f'Geography - {v_from}': 'Geography', f'Reference Product - {v_from}': 'Reference Product', f'Activity Name - {v_to}': 'Activity Name - new', f'Geography - {v_to}': 'Geography - new', f'Reference Product - {v_to}': 'Reference Product - new'}, inplace=True)
    df['Version from'] = v_from
    df['Version to'] = v_to
    return df

In [5]:
change_report_38_39 = load_change_report_annex('3.8', '3.9')
change_report_39_391 = load_change_report_annex('3.9', '3.9.1')
change_report_391_310 = load_change_report_annex('3.9.1', '3.10')
mapping = pd.read_csv("energyscope_data/mapping_linked.csv")

In [59]:
change_report = pd.concat([change_report_38_39, change_report_39_391, change_report_391_310])

In [60]:
change_report = change_report.drop_duplicates()

In [64]:
change_report = change_report.drop(change_report[
    (change_report['Reference Product - new'] == change_report['Reference Product'])
    & (change_report['Activity Name - new'] == change_report['Activity Name'])
    & (change_report['Geography - new'] == change_report['Geography'])
].index)

In [65]:
changed_activities = [list(e) for e in {tuple(item) for item in change_report[['Reference Product', 'Activity Name', 'Geography']].values.tolist()}]

In [66]:
def update_mapping_file(mapping, change_report, changed_activities):
    
    updated_mapping = pd.DataFrame(data=[], columns=mapping.columns)
    counter = 0
    
    for i in range(len(mapping)):
        
        activity_name = mapping['Activity'].iloc[i]
        activity_prod = mapping['Product'].iloc[i]
        activity_geo = mapping['Location'].iloc[i]
        
        if [activity_prod, activity_name, activity_geo] in changed_activities:
            
            counter += 1
            
            activity_name_new, activity_prod_new, activity_geo_new = change_report[
                (change_report['Reference Product'] == activity_prod) 
                & (change_report['Activity Name'] == activity_name) 
                & (change_report['Geography'] == activity_geo)
            ][['Activity Name - new', 'Reference Product - new', 'Geography - new']].iloc[0]
            
            tech_name = mapping['Name'].iloc[i]
            tech_type = mapping['Type'].iloc[i]
            database = mapping['Database'].iloc[i]
            updated_mapping.loc[i] = [tech_name, tech_type, activity_prod_new, activity_name_new, activity_geo_new, database]
            
            print(f"Old: {activity_prod} - {activity_name} - {activity_geo}")
            print(f"New: {activity_prod_new} - {activity_name_new} - {activity_geo_new}")
            
        else:
            updated_mapping.loc[i] = mapping.iloc[i]
    
    return updated_mapping, counter

In [67]:
updated_mapping, counter = update_mapping_file(mapping, change_report, changed_activities)
while counter > 0:
    updated_mapping, counter = update_mapping_file(updated_mapping, change_report, changed_activities)
    print(counter)

Old: methanol - methanol production - GLO
New: methanol - methanol production, natural gas reforming - CN
0
